In [1]:
import pandas as pd
import screed
from collections import Counter
import gzip
import pickle

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.glyphs import Circle
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool, BoxZoomTool, ResetTool, CustomJS, CrosshairTool

#%matplotlib inline

In [2]:
output_notebook()

Loading BokehJS ...

In [2]:
ir_df = pd.read_csv("data/mapping_file.txt", delimiter=",")

In [3]:
ir_df.head()

,Sample,i1_r,i2_f
0,8106_PC,cgagagtt,atcgtacg
1,8533_TF,cgagagtt,actatctg
2,8534_G,cgagagtt,tagcgagt
3,8535_PC,cgagagtt,ctgcgtgt
4,8508_TF,cgagagtt,tcatcgag


In [4]:
!zcat /var/seq_data/echinoderm/index1.fq.gz | head -n2

@M02465:260:000000000-AN086:1:1101:15603:1331 1:N:0:0
ACTCACTG

gzip: stdout: Broken pipe


In [5]:
!zcat /var/seq_data/echinoderm/read1.fq.gz | head -n2

@M02465:260:000000000-AN086:1:1101:15603:1331 1:N:0:0
TACGGAGGGTCCGAGCGTTATCCGGATTCATTGGGTTTAAAGGGTGCGTAGGCGGCTTCATAAGCCAGTGGTGAAAGCTTGCAGCTTAACTGTAAAATTGCCATTGGAACTGTAGAGCTTGAGTATGATTGAAGTAGGCGGAATAAATAGTGTAGCGGTGAAATGCTTAGATATTATTTAGAACACCAATTGCGAAGGCAGCTTACTAAGTCATAACTGACGATGATGCACGAAAGCGTGGGTAGCGAACA

gzip: stdout: Broken pipe


In [6]:
read1_fhs = dict([(s, gzip.open("data/demult/%s.R1.fq.gz"%s, "w")) for s in ir_df["Sample"]])
read2_fhs = dict([(s, gzip.open("data/demult/%s.R2.fq.gz"%s, "w")) for s in ir_df["Sample"]])

idx_d = dict([(row["i1_r"].upper() + row["i2_f"].upper(), row["Sample"]) for i, row in ir_df.iterrows()])

path = "/var/seq_data/echinoderm/"
idx_r1 = screed.open(path + "index1.fq.gz")
idx_r2 = screed.open(path + "index2.fq.gz")
read1 = screed.open(path + "read1.fq.gz")
read2 = screed.open(path + "read2.fq.gz")

seq_cnt = Counter()
bc_cnt = Counter()
bad_bc_count = 0

In [ ]:
for i, recs in enumerate(zip(idx_r1, idx_r2, read1, read2)):
    ir1 = recs[0]
    ir2 = recs[1]
    r1 = recs[2]
    r2 = recs[3]
        
    idx_key = ir1.sequence + ir2.sequence
    bc_cnt[idx_key] += 1
    
    try:
        sample_name = idx_d[idx_key]
    except KeyError:
        bad_bc_count += 1
        continue
    
    seq_cnt[sample_name] += 1
    
    fastq1 = "@%s\n%s\n+\n%s\n"%(r1.name, r1.sequence, r1.quality)
    read1_fhs[sample_name].write(fastq1.encode())

    fastq2 = "@%s\n%s\n+\n%s\n"%(r2.name, r2.sequence, r2.quality)
    read2_fhs[sample_name].write(fastq2.encode())

In [18]:
for i, j in zip(read1_fhs.values(), read2_fhs.values()):
    i.close(), j.close()

In [19]:
print("We found %s barcodes that did not match the mapping file."%str(bad_bc_count))

We found 2169313 barcodes that did not match the mapping file.


In [21]:
pickle.dump(seq_cnt, open("data/seq_cnt.pickle", "wb"))
pickle.dump(bc_cnt, open("data/bc_cnt.pickle", "wb"))

In [3]:
seq_cnt = pickle.load(open("data/seq_cnt.pickle", "rb"))

## Code below does not run

In [4]:
df = pd.DataFrame.from_dict(dict(seq_cnt.most_common()), orient="index")
df.rename(columns={0 : "count"}, inplace=True)
df.sort_values("count", inplace=True, ascending=False)
df["color"] = "#F8766D"
df.reset_index(inplace=True)
df.rename(columns={"index" : "SampleID"}, inplace=True)
df["x"] = [i + 1 for i, s in enumerate(df["SampleID"])]
df.head()

,SampleID,count,color,x
0,C227_W,226081,#F8766D,1
1,USP13A_CF,220694,#F8766D,2
2,C560_PC,211524,#F8766D,3
3,C280_S,127067,#F8766D,4
4,C340_CF,110776,#F8766D,5


In [8]:
p = figure(width=800, height=400, y_axis_type = "log", 
           tools="", toolbar_location="above", 
           y_axis_label = "Seq count", x_axis_label = "Sample")

p.xaxis.axis_line_width = 3
p.yaxis.axis_line_width = 3
p.outline_line_color = None
p.grid.grid_line_color = None

source = ColumnDataSource(df)

invisible_circle = Circle(x='x', y='count', 
                          fill_color='color', 
                          fill_alpha=0.5, 
                          line_color="color", 
                          line_alpha = 0.5, size=8)

visible_circle = Circle(x='x', y='count', 
                        fill_color='color', 
                        fill_alpha=1.0, 
                        line_color="color")

cr = p.add_glyph(source, 
                 invisible_circle, 
                 selection_glyph=visible_circle, 
                 nonselection_glyph=invisible_circle)

l = p.line(x = df["x"], 
           y = df["count"], 
           line_width=3, 
           color='#F8766D')

tooltip = """
    <div>
        <span style="font-size: 17px; font-weight: bold;">@SampleID </span>
    </div>
    <div>
        <span style="font-size: 17px; font-weight: bold;">@count </span>
    </div>
"""

code = "source.set('selected', cb_data['index']);"
callback = CustomJS(args={'source': source}, code=code)

p.add_tools(HoverTool(tooltips=tooltip, callback=callback, renderers=[cr]),
            BoxZoomTool(),
            ResetTool())

show(p)

In [9]:
!mv data/demult/C209.3_TF.R1.fq.gz data/demult/C209_3_TF.R1.fq.gz
!mv data/demult/C209.3_TF.R2.fq.gz data/demult/C209_3_TF.R2.fq.gz
!mv data/demult/C209.4_TF.R1.fq.gz data/demult/C209_4_TF.R1.fq.gz
!mv data/demult/C209.4_TF.R2.fq.gz data/demult/C209_4_TF.R2.fq.gz

In [10]:
seq_cnt.most_common(10)

[('C227_W', 226081),
 ('USP13A_CF', 220694),
 ('C560_PC', 211524),
 ('C280_S', 127067),
 ('C340_CF', 110776),
 ('C181_TF', 108153),
 ('8508_CF', 105777),
 ('USF6B_CF', 96555),
 ('C542_S', 95096),
 ('USP27A_CF', 90164)]

In [11]:
df[df["count"] < 1000].shape

(22, 4)